In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/20180418_gRNA1_Pax7_Laminin/sec/pax7counts'

In [6]:
# Add experiment date here to apply to dataframe
expt_date = '20180418'
treatment = 'SMPD3gRNA1'
result_type = 'Pax7Counts'

path = os.path.abspath('')+'/pax7countCSV/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Parse Image name
    df['Target'], df['Treatment'], df['Stain1'], df['Stain2'], df['Stain3'], df['Embryo'], df['Section'], df['MIP'] = zip(*df['Image'].map(lambda x: x.split('_')))

    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Slice,Count,Total Area,Average Size,%Area,Mean,IntDen,Date,Image,Target,Treatment,Stain1,Stain2,Stain3,Embryo,Section,MIP
0,CntlSide,23,1227.679,53.377,33.703,255,13611.219,20180418,SMPD3_gRNA1_Pax7_Ncad_Laminin_Emb7_sec3_MIP,SMPD3,gRNA1,Pax7,Ncad,Laminin,Emb7,sec3,MIP
1,ExptSide,16,1118.489,69.906,39.387,255,17825.913,20180418,SMPD3_gRNA1_Pax7_Ncad_Laminin_Emb7_sec3_MIP,SMPD3,gRNA1,Pax7,Ncad,Laminin,Emb7,sec3,MIP
0,CntlSide,23,1297.603,56.418,27.983,255,14386.473,20180418,SMPD3_gRNA1_Pax7_Ncad_Laminin_Emb7_sec2_MIP,SMPD3,gRNA1,Pax7,Ncad,Laminin,Emb7,sec2,MIP
1,ExptSide,23,1901.472,82.673,34.212,255,21081.537,20180418,SMPD3_gRNA1_Pax7_Ncad_Laminin_Emb7_sec2_MIP,SMPD3,gRNA1,Pax7,Ncad,Laminin,Emb7,sec2,MIP
0,CntlSide,16,2231.258,139.454,17.749,255,35560.668,20180418,SMPD3_gRNA1_Pax7_Ncad_Laminin_Emb6_sec3_MIP,SMPD3,gRNA1,Pax7,Ncad,Laminin,Emb6,sec3,MIP


In [7]:
# Mean counts across sections
mean_sections = pd.DataFrame((full_df.groupby(['Embryo', 'Slice'])['Count']).mean())
mean_sections
# Extract means for Cntl and Expt counts
pax7_cntl = mean_sections.xs('CntlSide', level='Slice')['Count'] 
pax7_expt = mean_sections.xs('ExptSide', level='Slice')['Count']

# Generate ratios as Expt/Cntl
pax7_ratios = pd.DataFrame(pax7_expt / pax7_cntl)
pax7_ratios.columns = ['Expt/Cntl Pax7']

In [8]:
# Combine processed values into single dataframe and output as csv file
pax7_cntl = pd.DataFrame(pax7_cntl)
pax7_cntl.columns = ['Cntl Pax7']
pax7_expt = pd.DataFrame(pax7_expt)
pax7_expt.columns = ['Expt Pax7']

results = pd.concat([pax7_cntl, pax7_expt, pax7_ratios], axis=1, sort=True)
results['ID'] = expt_date + '_' + results.index
results.to_csv(expt_date + '_' + treatment + '_' + result_type + '_results.csv')